In [ ]:
import os
import json
import glob 

import numpy as np
import pandas as pd

import cv2

In [ ]:
datasets_dir = os.path.abspath("../data/tator/grid/datasets")
datasets_dir

datasets_folders = glob.glob(os.path.join(datasets_dir, "*"))
datasets_folders

In [ ]:
dataset = datasets_folders[0]
images = glob.glob(os.path.join(dataset, "*.png"))
json_files = glob.glob(os.path.join(dataset, "*.json"))

In [ ]:
json_data = []

for json_file in json_files:
    with open(json_file, 'r') as f:
        d = json.load(f)
        json_data.append(d)

In [ ]:
json_file, json_data[0]['grid'].keys()

In [ ]:
json_data[0]['grid']

In [ ]:
cell_data = []

for j in json_data:
    keys = j['grid'].keys()
    for k in keys:
        d = j['grid'][k]
        for sub_key in d.keys():
            cell = d[sub_key]
            if not len(cell['labels']):
                continue
            
            label = cell['labels'][0]
            top_left_x = cell['top_left_x']
            top_left_y = cell['top_left_y']
            width = cell['width']
            height = cell['height']
            
            col = cell['center_x']
            row = cell['center_y']
            patch_size = min(width, height)
            # Split off the "_version*.json" part of the filename
            image_name = os.path.basename(json_file).split("_version")[0] + ".png"
            cell_data.append([image_name, label, col, row, patch_size])

In [ ]:
pd.DataFrame(cell_data, columns=["image_name", "label", "col", "row", "patch_size"])

In [ ]:
import matplotlib.pyplot as plt

# Get unique labels and assign colors
unique_labels = list(set([cell[1] for cell in cell_data]))
color_map = {label: plt.cm.tab10(i % 10) for i, label in enumerate(unique_labels)}

# plot all of the coordinates, and create colored rectangles around each point, based on patch size
plt.figure(figsize=(10, 10))
img = cv2.imread(os.path.join(dataset, images[0]))
plt.imshow(img)
for i, cell in enumerate(cell_data):
    image_name, label, col, row, patch_size = cell
    color = color_map[label]
    rect = plt.Rectangle((col - patch_size / 2, row - patch_size / 2), patch_size, patch_size, linewidth=1, edgecolor=color, facecolor='none')
    plt.gca().add_patch(rect)
    plt.text(col, row, "", color=color, fontsize=8, ha='center')